In [72]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, mean_squared_error

In [73]:
submission = pd.read_csv('submission/submission.csv')
M_results = pd.read_csv('submission/MNCAATourneyDetailedResults.csv')
W_results = pd.read_csv('submission/WNCAATourneyDetailedResults.csv')

In [74]:
submission

,ID,Pred
0,2023_1104_1112,0.723990
1,2023_1104_1113,0.915681
2,2023_1104_1116,0.845466
3,2023_1104_1120,0.845042
4,2023_1104_1124,0.839218
...,...,...
4551,2023_3437_3450,0.472925
4552,2023_3437_3452,0.721032
4553,2023_3439_3450,0.697753
4554,2023_3439_3452,0.925963


In [75]:
def duplicate_and_switch_sides(df):
    # locationL
    #   - 1 if T1 plays home court
    #   - -1 if T1 plays away court
    #   - 0 if neutral court 
    df_copy = df.copy()
    df_copy["location"] = df_copy["WLoc"].apply(lambda x: "H" if x == "A" else "A" if x == "H" else "N")
    df_copy.drop(columns=["WLoc"], inplace=True)
    df_copy.columns = df_copy.columns.str.replace('W', 'T2_').str.replace('L', 'T1_')
    
    df["location"] = df["WLoc"]
    df.drop(columns=["WLoc"], inplace=True)

    df.columns = df.columns.str.replace('W','T1_').str.replace('L','T2_')
    
    
    result = pd.concat([df, df_copy],axis=0)
    result["location"] = result["location"].apply(lambda x: 1 if x == "H" else 0 if x == "N" else -1)
    result["point_diff"] = result["T1_Score"] - result["T2_Score"]
    result.sort_values(by=["Season", "T1_TeamID", "T2_TeamID"], inplace=True)
    result.reset_index(drop=True, inplace=True)
    return result   
    

In [76]:
M_new = duplicate_and_switch_sides(M_results).query("Season == 2023 and T1_TeamID < T2_TeamID")
W_new = duplicate_and_switch_sides(W_results).query("Season == 2023 and T1_TeamID < T2_TeamID")

In [77]:
M_new["win"] = np.array(M_new["point_diff"] > 0, dtype=int)
W_new["win"] = np.array(W_new["point_diff"] > 0, dtype=int)

In [78]:
M_new["ID"] = M_new["Season"].astype(str) + "_" + M_new["T1_TeamID"].astype(str) + "_" + M_new["T2_TeamID"].astype(str)
W_new["ID"] = W_new["Season"].astype(str) + "_" + W_new["T1_TeamID"].astype(str) + "_" + W_new["T2_TeamID"].astype(str)

In [79]:
M_final = M_new[["ID", "win"]]
W_final = W_new[["ID", "win"]]

In [52]:
results = pd.concat([M_final, W_final], axis=0).reset_index(drop=True)

In [56]:
final = results.merge(submission, on="ID", how="left")

In [66]:
mean_squared_error(final["win"], final["Pred"])

0.19361362768443618

In [65]:
accuracy_score(final["win"], round(final["Pred"]))

0.7142857142857143

In [80]:
W_final = W_final.merge(submission, on="ID", how="left")
M_final = M_final.merge(submission, on="ID", how="left")

In [81]:
mean_squared_error(W_final["win"], W_final["Pred"])

0.16965865097725974

In [82]:
accuracy_score(W_final["win"], round(W_final["Pred"]))

0.746031746031746

In [83]:
mean_squared_error(M_final["win"], M_final["Pred"])

0.2175686043916126

In [84]:
accuracy_score(M_final["win"], round(M_final["Pred"]))


0.6825396825396826